# HPH第二版实验

### 思路

*id统计相关*
1. item_id, shop_id的交易量, 交易率
2. item_id, shop_id7天是否新出现, 3天是否新出现, 昨天是否新出现


In [99]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
offline_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt', sep=' ')
# online_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt', sep=' ')

# 1 数据集统一处理

In [100]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

offline_df['date'] = offline_df['context_timestamp'].apply(lambda x: extract_date(x))
offline_df['weekday'] = offline_df['context_timestamp'].apply(lambda x: extract_weekday(x))
offline_df['hour'] = offline_df['context_timestamp'].apply(lambda x: extract_hour(x))

In [101]:
%run ../util/time_utils.py

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 3)

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 1)


In [102]:
# 统计信息附加

def getColTradeRate(df, idCol):
    rateCol = idCol + '_tr'
    pvCol = idCol + '_pv'
    try:
        del df[rateCol]
        del df[pvCol]
    except:
        pass
    a = offline_df.groupby([idCol]).agg({'is_trade':'sum'})
    b = offline_df.groupby([idCol]).agg({'is_trade':'size'})
    c = a.join(b, lsuffix="_sum", rsuffix="_size")
    c[rateCol] = c['is_trade_sum'] / c['is_trade_size']
    c[pvCol] = c['is_trade_size']
    return offline_df.join(c[[rateCol, pvCol]], on=idCol)
    

# 各类id的 交易量, 交易率
offline_df = getColTradeRate(offline_df, 'item_city_id')
offline_df = getColTradeRate(offline_df, 'item_id')
offline_df = getColTradeRate(offline_df, 'shop_id')
offline_df = getColTradeRate(offline_df, 'item_brand_id')
offline_df = getColTradeRate(offline_df, 'user_occupation_id')


In [103]:
# 分桶
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

offline_df['item_city_id_pv_cut'] = pd.cut(offline_df['item_city_id_pv'].values, bins=10, retbins=False)
offline_df['item_id_pv_cut'] = pd.cut(offline_df['item_id_pv'].values, bins=10, retbins=False)
offline_df['shop_id_pv_cut'] = pd.cut(offline_df['shop_id_pv'].values, bins=10, retbins=False)
offline_df['item_brand_id_pv_cut'] = pd.cut(offline_df['item_brand_id_pv'].values, bins=10, retbins=False)
offline_df['user_occupation_id_pv_cut'] = pd.cut(offline_df['user_occupation_id_pv'].values, bins=10, retbins=False)

offline_df['item_city_id_pv_cut'] = le.fit_transform(offline_df['item_city_id_pv_cut'].copy())
offline_df['item_id_pv_cut'] = le.fit_transform(offline_df['item_id_pv_cut'].copy())
offline_df['shop_id_pv_cut'] = le.fit_transform(offline_df['shop_id_pv_cut'].copy())
offline_df['item_brand_id_pv_cut'] = le.fit_transform(offline_df['item_brand_id_pv_cut'].copy())
offline_df['user_occupation_id_pv_cut'] = le.fit_transform(offline_df['user_occupation_id_pv_cut'].copy())

In [104]:
# standardization and scaling

from sklearn import preprocessing

def scale(df, idCol):
    min_max_scaler = preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df[idCol].values.reshape(-1, 1))
    return pd.Series(a.reshape(1, -1)[0])

# 令数据再少一些?
offline_df['shop_score_service_scaled'] = scale(offline_df, 'shop_score_service')
offline_df['shop_score_delivery_scaled'] = scale(offline_df, 'shop_score_delivery')
offline_df['shop_score_description_scaled'] = scale(offline_df, 'shop_score_description')
offline_df['shop_review_positive_rate_scaled'] = scale(offline_df, 'shop_review_positive_rate')
offline_df['item_city_id_tr_scaled'] = scale(offline_df, 'item_city_id_tr')
offline_df['item_id_tr_scaled'] = scale(offline_df, 'item_id_tr')
offline_df['shop_id_tr_scaled'] = scale(offline_df, 'shop_id_tr')
offline_df['item_brand_id_tr_scaled'] = scale(offline_df, 'item_brand_id_tr')
offline_df['user_occupation_id_scaled'] = scale(offline_df, 'user_occupation_id_tr')

# offline_df = offline_df.dropna()

# 2 生成矩阵数据

In [105]:
used = 78261
X = offline_df[
    [
       'date', 
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
]
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']]

# 3 kfold 测试

In [158]:
train_input = X
train_input['is_trade'] = y
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost
%run ../yuhua/BaseFrame.py

score_func = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

it = 0
def tatx(train_df, test_df, valid_df):

    X_train = train_df.copy()
    del X_train['is_trade']

    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    X_valid = valid_df.copy()
    del X_valid['is_trade']
    y_valid = valid_df['is_trade']
    
    if it == 0:
        xgb = xgboost.XGBClassifier()
        parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}
        gs = GridSearchCV(xgb, parameters, n_jobs=5, 
    else:
       
    params = {
        'nthread': 20
    }
    m = xgboost.XGBClassifier(**params)
    m.fit(X_train, y_train)

    
    l_train = log_loss(y_train, m.predict_proba(X_train))
    l_test = log_loss(y_test, m.predict_proba(X_test))
    l_valid = log_loss(y_valid, m.predict_proba(X_valid))

    return l_train, l_test, l_valid

frame = BaseFrame(train_input, 0.05)
frame.kfold_by_date('date', tatx)
# 0.077189, 0.076092
# test = 0.077047  valid = 0.07597
# test = 0.077077  valid = 0.075944

/Users/prehawkmac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2018-09-18 to 2018-09-18 is training set, 2018-09-19 is test set, 2018-09-24 is valid set, start training ... 
0 done, logloss train = 0.08453301421 (compare to previous: 0.08453301421), 
                    	test = 0.0860219639715 (compare to previous: 0.0860219639715), 
                    	valid = 0.0764235695799 (compare to previous: 0.0764235695799)

2018-09-18 to 2018-09-19 is training set, 2018-09-20 is test set, 2018-09-24 is valid set, start training ... 
1 done, logloss train = 0.0850018344811 (compare to previous: 0.000468820271079), 
                    	test = 0.0850253621848 (compare to previous: -0.00099660178662), 
                    	valid = 0.0762046606052 (compare to previous: -0.000218908974612)

2018-09-18 to 2018-09-20 is training set, 2018-09-21 is test set, 2018-09-24 is valid set, start training ... 
2 done, logloss train = 0.0849235743849 (compare to previous: -7.82600961871e-05), 
                    	test = 0.0842971726149 (compare to previous: -0.000728189

In [113]:
indices = np.arange(10)
indices[2:7]

array([2, 3, 4, 5, 6])

In [131]:
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.metrics import make_scorer


import xgboost
% run ../util/time_series_split.py


dtsv = DateTimeSplit(dateSeries=offline_df['context_timestamp'], fmt="%Y-%m-%d")

for train_i, test_i in dtsv.split(X, y):
    X_train, y_train = X.values[train_i], y.values[test_i]
    X_test, y_test = X.values[test_i], y.values[test_i]
    
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    
    
    
    

build df done
(78268, 38) (70931, 1)
(70931, 38) (70931, 1)
(149199, 38) (68387, 1)
(68387, 38) (68387, 1)
(217586, 38) (71199, 1)
(71199, 38) (71199, 1)
(288785, 38) (68318, 1)
(68318, 38) (68318, 1)
(357103, 38) (63614, 1)
(63614, 38) (63614, 1)


In [129]:
indices = np.arange(len(X))
indices[ind]
X.values[ind]

array([['2018-09-18', 3412720377098676069, 1975590437749032870, ..., 1.0,
        1.0, 1.0],
       ['2018-09-18', 3412720377098676069, 1975590437749032870, ..., 1.0,
        1.0, 1.0],
       ['2018-09-18', 3412720377098676069, 1975590437749032870, ..., 1.0,
        1.0, 1.0],
       ..., 
       ['2018-09-18', 5251805314991883348, 7838285046767229711, ...,
        0.9518072289156628, 0.953125, 0.9556626506024096],
       ['2018-09-18', 5251805314991883348, 7838285046767229711, ...,
        0.9518072289156628, 0.953125, 0.9556626506024096],
       ['2018-09-18', 5251805314991883348, 7838285046767229711, ...,
        0.9518072289156628, 0.953125, 0.9556626506024096]], dtype=object)